In [1]:
from ROOT import TH1D, TCanvas, TF1, std
from pyLCIO import EVENT, UTIL, IOIMPL, IMPL
import matplotlib.pyplot as plt
import numpy as np
import h5py 
import awkward1 as ak 
import pandas as pd
plt.rcParams.update({'font.size': 17})

##conda install -c anaconda pytables

from functions import CellIDDecoder
from scipy.stats import norm

Welcome to JupyROOT 6.22/00
Loading LCIO ROOT dictionaries ...


In [2]:
fName="angle/photon-5deg-z1cm.slcio"
collection = 'ECalBarrelSiHitsOdd'
reader = IOIMPL.LCFactory.getInstance().createLCReader()
reader.open( fName )

In [3]:
maxEvt = 1
nEvt = 0

df = pd.DataFrame(columns=['layer', 'x', 'y', 'z', 'e'])
nhit = 0
for evt in reader:
    nEvt += 1
    if nEvt > maxEvt:
            break
    
    ecalBarrel = evt.getCollection(collection)
    cellIDString = ecalBarrel.getParameters().getStringVal("CellIDEncoding")
    decoder = CellIDDecoder( cellIDString ) 
    for hit in ecalBarrel:
        nhit += 1  
        l = decoder.layer( hit.getCellID0() ) 
        e = hit.getEnergy()
        pos = hit.getPosition()
        df.loc[nhit] = [l,pos[0], pos[1], pos[2], e]
        #print (l,pos[0], pos[1], pos[2], e)
        
        

In [4]:
df

,layer,x,y,z,e
1,5.0,10.061804,1839.667480,128.344162,0.002828
2,7.0,12.856804,1852.137451,128.344162,0.007597
3,9.0,10.563471,1864.607544,128.344162,0.019933
4,11.0,8.270137,1877.077515,128.344162,0.019514
5,3.0,12.355137,1827.197510,138.520828,0.000095
...,...,...,...,...,...
417,11.0,8.270137,1877.077515,118.167503,0.000164
418,9.0,0.386804,1864.607544,153.785828,0.000016
419,9.0,-25.054863,1864.607544,184.315826,0.000876
420,17.0,84.803467,1914.487549,184.315826,0.000171


In [5]:
df.to_hdf('data.hdf5', key='shower', mode='w')

In [6]:
f = h5py.File('data.hdf5', 'r')

In [29]:
a = f['shower']['block0_values']

In [34]:
a[303]

array([1.70000000e+01, 1.25668039e+01, 1.91448755e+03, 2.10757507e+02,
       4.94790031e-04])